In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125315")
exp = Experiment(workspace=ws, name="quick-starts-ws-124627")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125315
Azure region: southcentralus
Subscription id: dafd62fe-51d3-4671-af50-6bac2f001b5d
Resource group: aml-quickstarts-125315


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###cpu_cluster_name = "cpucluster"

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
Cpu_Cluster = ComputeTarget.create(ws, "CpuCluster", compute_config)

Cpu_Cluster.wait_for_completion(show_output=True)


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform

import os


# Specify parameter sampler
ps = RandomParameterSampling( {
        "keep_probability": uniform(0.05, 0.1),
        
    
    }
) ### YOUR CODE HERE ###


# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=2,slack_factor=0.2) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
from azureml.core import ScriptRunConfig
from azureml.core import Environment

#sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
#src = ScriptRunConfig(source_directory="~/cloudfiles/code/Users/odl_user_124627",
 #                     script='train.py',
  #                    compute_target=Cpu_Cluster
   #                 ) ### YOUR CODE HERE ###
#script_run_config.run_config.target = Cpu_Cluster
#est=SKLearn(src)
src="~/cloudfiles/code/Users/odl_user_124627"
script='train.py'
est=SKLearn(source_directory='.',entry_script=script,compute_target=Cpu_Cluster)



# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=early_termination_policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=4,
                             max_concurrent_runs=4
                                    )
                              ### YOUR CODE HERE ###

In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run=exp.submit(hyperdrive_config,show_output=True )
RunDetails(hyperdrive_run).show() #Run history widget
hyperdrive_run.wait_for_completion(show_output=True)
#best_run = hyperdrive_run.get_best_run_by_primary_metric()
#best_run_metrics = best_run.get_metrics()
#parameter_values = best_run.get_details()['runDefinition']['Arguments']

#print('\n Accuracy:', best_run_metrics['accuracy'])


### YOUR CODE HERE ###

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ed7ae342-a369-408c-bcf5-be52eabf8b33
Web View: https://ml.azure.com/experiments/quick-starts-ws-124627/runs/HD_ed7ae342-a369-408c-bcf5-be52eabf8b33?wsid=/subscriptions/dafd62fe-51d3-4671-af50-6bac2f001b5d/resourcegroups/aml-quickstarts-125315/workspaces/quick-starts-ws-125315

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-08T22:48:20.259609][API][INFO]Experiment created<END>\n""<START>[2020-11-08T22:48:20.777344][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-08T22:48:21.017148][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-08T22:48:21.8506163Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_ed7ae342-a369-408c-bcf5-be52eabf8b33
Web View: https://ml.azure.com/experiments/quick-starts-ws-124627/runs/HD_ed7ae342-a369-408c-bcf5-be52eabf8b33?wsid=/subsc

{'runId': 'HD_ed7ae342-a369-408c-bcf5-be52eabf8b33',
 'target': 'CpuCluster',
 'status': 'Canceled',
 'startTimeUtc': '2020-11-08T22:48:20.032424Z',
 'endTimeUtc': '2020-11-08T22:58:19.815962Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": {\n      "type": null,\n      "message": "[(https://southcentralus.experiments.azureml.net/subscriptions/dafd62fe-51d3-4671-af50-6bac2f001b5d/resourceGroups/aml-quickstarts-125315/providers/Microsoft.Machine

In [12]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run.get_details()['runDefinition']['Arguments'])
print('Best Run Id: ', best_run.id)
model_hd=best_run.register_model(model_name='log_reg_with_hyperdrive',outputpath='outputs/model_hd')

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [31]:
from azureml.data.dataset_factory import TabularDatasetFactory


# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
src="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds=TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [32]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###

TypeError: 'NoneType' object is not iterable

In [33]:
#splitting the data
from sklearn.model_selection import train_test_split  
train_data, test_data = train_test_split(ds, test_size=0.2, random_state=42) 

NameError: name 'x' is not defined

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="logistic-regression",
    primary_metric="Accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=3)

In [2]:
# Submit your automl run
ws = Workspace.from_config()
experiment = Experiment(ws, "exp")
run = experiment.submit(automl_config, show_output=True)

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.
model_automl=run.register_model(model_name='log_reg_automl',outputpath='outputs/model_automl')
### YOUR CODE HERE ###